In [6]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
import numpy as np
import pandas as pd
import matplotlib as ml
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import playercareerstats
years="2000-01"
seasonType='Regular Season'
allTeamId=pd.DataFrame(teams.get_teams())['id']


In [11]:
from nba_api.stats.endpoints import homepageleaders
test=homepageleaders.HomePageLeaders(player_or_team="team",season="2000-01",season_type_playoffs=seasonType).get_data_frames[2]
test

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
from nba_api.stats.endpoints import playercareerstats
career = playercareerstats.PlayerCareerStats(player_id='203999').get_data_frames()[0]

# pandas data frames (optional: pip install pandas)
career


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203999,2015-16,00,1610612743,DEN,21.0,80,55,1733.0,307,...,0.811,181,379,560,189,79,50,104,208,796
1,203999,2016-17,00,1610612743,DEN,22.0,73,59,2038.0,494,...,0.825,212,506,718,359,61,55,171,214,1221
2,203999,2017-18,00,1610612743,DEN,23.0,75,73,2443.0,504,...,0.850,195,608,803,458,90,61,210,212,1385
3,203999,2018-19,00,1610612743,DEN,24.0,80,80,2504.0,616,...,0.821,228,637,865,580,108,55,248,228,1604
4,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335.0,565,...,0.817,166,545,711,512,85,44,226,222,1456
5,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488.0,732,...,0.868,205,575,780,599,95,48,222,192,1898
6,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476.0,764,...,0.810,206,813,1019,584,109,63,281,191,2004
7,203999,2022-23,00,1610612743,DEN,27.0,3,3,107.0,25,...,0.864,6,26,32,29,4,1,11,10,72


# Four Factors

In [ ]:
from nba_api.stats.endpoints import boxscorefourfactorsv2
from nba_api.stats.endpoints import boxscoresummaryv2
df=boxscorefourfactorsv2.BoxScoreFourFactorsV2(game_id='0022100406' ).get_data_frames()[1]
# eliminateList=[  x for x in df
#     if  x!="GAME_ID" and  x!="TEAM_ID" and  x!="TEAM_ABBREVIATION" and x!="TEAM_CITY" and x!="MIN" ]
# df.loc[:,eliminateList]
idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id='0022100001').get_data_frames()[7]

# EFG_PCT=[(All Field Goals Made) + 0.5*(3P Field Goals Made)]/(All Field Goal Attempts) (effective)
# OREB_PCT=(Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]
#FTA_RATE= (Total Free Throws Attempted) / (Total Field Goal Attempted)
# TM_TOV_PCT=(Turnovers)*100)/ [(Field Goal Attempts)+(Free Throw Attempts*0.44)+(Assists)+(Turnovers)]
#OPP_OREB_PCT=DEFENSIVE REBOUNDING PERECENTAGE%
idHomeTeam


In [8]:
count=0
tmp=[]
nameFile="\\MatchId"+years+".txt"
for tmId in allTeamId:
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=tmId,season_nullable=years,season_type_nullable=seasonType).get_data_frames()[0]
    
    if(count==0):
        tmp=df["GAME_ID"].tolist()
        count=1
    else:
        for x in df["GAME_ID"].tolist():
            if x not in tmp:
                tmp.append(x)
            

with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'w') as f:
            for line in tmp:
                tmpS=str(line)
                f.write(tmpS)
                f.write('\n')







In [9]:
MatchId=[]
nameFile="/MatchId"+years+".txt"

with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile,'r') as f:
        fr=f.readlines()  
        for line in fr:
            line=line.replace('\n', '')
           
            # We take the first 650 match 
            if(int(line[-3:]) <=650 and int(line[-4])==0):
                MatchId.append(line)

chunk_size = 50
chunked_list = [MatchId[i:i+chunk_size] for i in range(0, len(MatchId), chunk_size)]



# TRADIONAL SPLIT

In [10]:
from nba_api.stats.endpoints import boxscoretraditionalv2 
from nba_api.stats.endpoints import boxscoresummaryv2
import winsound

tmpHomeTeam=[]
columns_count=0
BoxScores=[]
nameFile="\\BoxScore"+years+"traditional"+".txt"


for i in range(len(chunked_list)):
   
    tmpBox=[]
    for x in chunked_list[i]:    

        df=boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=x).get_data_frames()[1]
        idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id=x)
        
        


        df["HOME"]=0

        label=[2]
        if(df.at[0,'PTS'] > df.at[1,'PTS']):
            label=[1]

        if(df.at[0,"TEAM_ID"]==idHomeTeam):
            df.at[0,'HOME']=1
        else:
            df.at[1,'HOME']=1
        
        
        eliminateList=[  x for x in df
            if  x!="GAME_ID"  and  x!="TEAM_ABBREVIATION" and x!="TEAM_CITY"   and x!='PLUS_MINUS' and x!="TEAM_NAME" and x!='MIN' and # x!="TEAM_ID" and 
                x!='FGM'   and x!='FG3M'and x!='FTM' and x!='REB' # and  x!='FGA' and x!='FG3A' and x!='FTA' 
            ]
            
        df=df.loc[:,eliminateList]
       
        tmp=list(df.loc[0])+list(df.loc[1])+label 
        
        tmpBox.append(tmp)
   
    with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'a') as f:
        if columns_count==0:
            tmpS=list(df.columns)
            tmpS.extend([el+"_O" for el in tmpS ])
            tmpS=str(tmpS)
            tmpS=tmpS.replace("[","")
            tmpS=tmpS.replace("]","")
            f.write(tmpS)
            f.write('\n')
            columns_count+=1
        for line in tmpBox:
            tmpS=str(line)
            tmpS=tmpS.replace("[","")
            tmpS=tmpS.replace("]","")
            f.write(tmpS)
            f.write('\n')
    print("Finito il "+str(i)+"-esimo chunk ")

winsound.Beep(440, 2500)

Finito il 0-esimo chunk 
Finito il 1-esimo chunk 
Finito il 2-esimo chunk 
Finito il 3-esimo chunk 
Finito il 4-esimo chunk 
Finito il 5-esimo chunk 
Finito il 6-esimo chunk 
Finito il 7-esimo chunk 
Finito il 8-esimo chunk 
Finito il 9-esimo chunk 
Finito il 10-esimo chunk 
Finito il 11-esimo chunk 
Finito il 12-esimo chunk 


# ADVANCE SPLIT

In [12]:
from nba_api.stats.endpoints import boxscoreadvancedv2 
from nba_api.stats.endpoints import boxscoresummaryv2
import winsound

tmpHomeTeam=[]
BoxScores=[]
nameFile="\\BoxScore"+years+"advance"+".txt"
columns_count=0
for i in range(len(chunked_list)):
    
    tmpBox=[]
    
    for x in chunked_list[i]:    
 
        df=boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=x).get_data_frames()[1]
        idHomeTeam=boxscoresummaryv2.BoxScoreSummaryV2(game_id=x)
  
        


        df["HOME"]=0

        label=[2]
        if(df.at[0,'NET_RATING'] > df.at[1,'NET_RATING']):
            label=[1]

        if(df.at[0,"TEAM_ID"]==idHomeTeam):
            df.at[0,'HOME']=1
        else:
            df.at[1,'HOME']=1
        eliminateList=[  x for x in df
            if  x!="GAME_ID"  and  x!="TEAM_ABBREVIATION"   and x!="TEAM_NAME" and x!='MIN' and # x!="TEAM_ID" and 
                x!="TEAM_CITY" and x.startswith("E_")==False
            ]
        df=df.loc[:,eliminateList]
        
        
        tmp=list(df.loc[0])+list(df.loc[1])+label 
        
        
        tmpBox.append(tmp)
    
    with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'a') as f:
        if columns_count==0:
            tmpS=list(df.columns)
            tmpS.extend([el+"_O" for el in tmpS ])
            tmpS=str(tmpS)
            tmpS=tmpS.replace("[","")
            tmpS=tmpS.replace("]","")
            f.write(tmpS)
            f.write('\n')
            columns_count+=1
        for line in tmpBox:
            tmpS=str(line)
            tmpS=tmpS.replace("[","")
            tmpS=tmpS.replace("]","")
            f.write(tmpS)
            f.write('\n')
    print("Finito il "+str(i)+"-esimo chunk ")


winsound.Beep(440, 2500)

Finito il 0-esimo chunk 
Finito il 1-esimo chunk 
Finito il 2-esimo chunk 
Finito il 3-esimo chunk 
Finito il 4-esimo chunk 
Finito il 5-esimo chunk 
Finito il 6-esimo chunk 
Finito il 7-esimo chunk 
Finito il 8-esimo chunk 
Finito il 9-esimo chunk 
Finito il 10-esimo chunk 
Finito il 11-esimo chunk 
Finito il 12-esimo chunk 


# Reminder Advance Stats

In [ ]:

#TmPoss=0.5 * (FGA + 0.475 * FTA - ORB + TOV)
#Counter of team possession
#OppPoss=same of top, but for opposite team

#TS_PCT=Pts / (2 * (FGA + 0.475 * FTA))

#OFF_RATING= 100 / (TmPoss + OppPoss) * Pts

#DEF_RATING= 100 / (TmPoss + OppPoss) * OppPts
#NET_RATING=OFF_RATING -DEF_RATING
#Adjusts a team's scoring offense and defense for their pace of play

# df[df.columns.difference(["GAME_ID","TEAM_ID","TEAM_ABBREVIATION","TEAM_CITY","MIN"])]

In [24]:
year="19"
nameFile="\\PTS"+year
tot=0
with open(r"D:\UniSergio\BasketScripts\WaB\BoxScoresFile"+nameFile, 'r') as f:
    fr=f.readlines()
    for count,line in enumerate(fr):
        x=float(line)
        tot+=x
round(tot/count,1)

46.7